In [1]:
import numpy as np
import scipy.stats
import scipy.odr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from tqdm import tqdm
import functools
print = functools.partial(print, flush=True) # print doesn't happen til script ends so force it to flush... windows thing?
import pandas as pd


params180916 = ['data/CHIME_180916.J0158+65/chime_180916_fit_params_real.csv',
          'data/CHIME_180916.J0158+65/chime_180916_fit_params_ddm1.csv',
          'data/CHIME_180916.J0158+65/chime_180916_fit_params_ddm2.csv',
          'data/CHIME_180916.J0158+65/chime_180916_fit_params_ddm-half.csv']

paramerrs180916 = ['data/CHIME_180916.J0158+65/chime_180916_param_errors_real.csv',
             'data/CHIME_180916.J0158+65/chime_180916_param_errors_ddm1.csv',
             'data/CHIME_180916.J0158+65/chime_180916_param_errors_ddm2.csv',
             'data/CHIME_180916.J0158+65/chime_180916_param_errors_ddm-half.csv']

dataframes180916 = []
for param, paramerr in zip(params180916, paramerrs180916):
    frame = pd.merge(pd.read_csv(param),
                     pd.read_csv(paramerr),
                     how='left', on='name')
    frame = frame.set_index('name')
    dataframes180916.append(frame)
dataframes180916[0]

,amplitude,xo,yo,sigmax,sigmay,theta,drift (mhz/ms),angle,time_res,center_f,amp_error,xo_error,yo_error,sigmax_error,sigmay_error,angle_error,drift error (mhz/ms),red_chisq
name,,,,,,,,,,,,,,,,,,
13.0,200.000000,80.315696,64.004278,14.890111,1.693837,1.788828,-28.696642,1.788828,0.000983,600,10.517381,0.190396,0.764730,0.783025,0.089074,0.008571,1.164628,0.686863
14.0,652.599573,80.362373,64.339455,8.604327,-2.630985,2.180545,-9.101534,2.180545,0.000983,600,12.831800,0.105779,0.141836,0.169202,0.051741,0.009381,0.181874,0.452591
15.0,170.235161,140.322815,64.354499,18.819966,6.143563,1.924006,-17.245310,1.924006,0.000983,600,2.639356,0.134811,0.275748,0.291807,0.095249,0.008011,0.425674,2.793222
15.5,527.529669,176.365099,64.243880,3.039805,8.197538,0.506631,-11.456694,-1.064165,0.000983,600,5.769830,0.052325,0.080040,0.033248,0.089660,0.006650,0.179574,2.190109
16.0,471.367870,80.277935,64.090880,2.125115,4.835542,0.287827,-21.475653,-1.282969,0.000983,600,13.110688,0.068337,0.130051,0.059108,0.134496,0.021425,1.690406,0.614140
18.0,2712.600356,80.460995,64.335280,14.449545,3.211479,1.942930,-16.288749,1.942930,0.000983,600,35.544413,0.079226,0.177046,0.189344,0.042082,0.004333,0.208356,0.094759
19.0,1259.746447,80.379281,64.297830,8.573349,-2.708255,2.169715,-9.314804,2.169715,0.000983,600,24.116838,0.101968,0.138677,0.164135,0.051850,0.009501,0.190066,0.436231
21.0,315.111128,80.294710,64.265043,8.833778,3.815343,-0.987329,-9.631106,-0.987329,0.000983,600,7.098947,0.131003,0.172664,0.198969,0.085941,0.016916,0.354344,0.514910
23.0,4302.701500,80.481257,64.129712,13.979072,2.061264,1.814525,-25.567108,1.814525,0.000983,600,55.082906,0.050209,0.173789,0.178961,0.026388,0.002729,0.297920,0.045614
